In [29]:
from dotenv import load_dotenv 
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone
from pinecone import ServerlessSpec

In [30]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'fecd707c9671ee5ede37f9f3e86f5feb', 'Date': 'Wed, 21 Aug 2024 19:26:53 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [28]:
import json
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Dr. Johnson is amazing! Her lectures are clear and she really cares about her students.'},
 {'professor': 'Professor Mark Davis',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Great professor, but the exams are really tough. Pay attention in class!'},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Lectures can be dry, but she knows her stuff. Office hours are helpful.'},
 {'professor': 'Professor James Lee',
  'subject': 'Chemistry',
  'stars': 2,
  'review': 'Not the best at explaining concepts, but the textbook covers everything.'},
 {'professor': 'Dr. Rachel Miller',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Very passionate about biology. Her enthusiasm makes the subject interesting.'},
 {'professor': 'Professor David Wilson',
  'subject': 'History',
  'stars': 4,
  'review': "Interesting lectures, but make sure to do the readings

In [33]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append ({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [34]:
processed_data[0]

{'values': [0.013769841,
  -0.014166009,
  0.012545321,
  0.053638753,
  0.0045469287,
  -0.019556295,
  0.027827805,
  0.05450312,
  -0.0065487777,
  0.0073771295,
  0.027635723,
  -0.013181591,
  -0.048572604,
  0.030204812,
  -0.0049490994,
  0.027875824,
  -0.017635481,
  -0.012521312,
  0.041225486,
  0.04273813,
  0.0452592,
  -0.019844418,
  0.017695507,
  -0.04069726,
  -0.033182077,
  -0.052486263,
  0.01883599,
  0.015930757,
  0.0374799,
  0.00039429226,
  0.07371127,
  0.004183775,
  -0.017503425,
  -0.01886,
  -0.03901655,
  0.018451827,
  0.0002432907,
  0.021693202,
  -0.00047232537,
  -0.009520038,
  -0.013049535,
  -0.0065847933,
  -0.006560783,
  0.02272564,
  0.043098282,
  0.00039691836,
  0.00275817,
  0.0055583576,
  0.047516156,
  0.048764687,
  -0.020192565,
  0.013769841,
  0.022329472,
  -0.013877887,
  -0.024634449,
  -0.009375977,
  0.026387192,
  0.057768505,
  0.004799036,
  -0.04187376,
  0.05589571,
  -0.0038086155,
  -0.029508518,
  -0.011632934,
  0.00

In [35]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [36]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}